In [364]:
import numpy as np
import codecs
import xlwt
import os.path
import openpyxl as px
import string
import glob
from openpyxl.styles import Border, Side

In [418]:
def main():
#     spot_log_names = ["AgentNum", "WaitTimeMenSmallToiletList"] #set log names of spot that you want to make excel sheet
    spot_log_names = ["AgentNum",   "MovedNum", "SexList", "TotalRouteNumToAcceptanceList", "RouteCostToAcceptanceList", "RouteCostToAcceptanceBetweenCorridorAndCorridorList", "RouteCostToAcceptanceBetweenCorridorAndLivingSpaceList", "RouteCostToAcceptanceBetweenLivingSpaceAndLivingSpaceList", "TotalRouteNumToToiletList", "RouteCostToToiletList", "RouteCostToToiletBetweenCorridorAndCorridorList", "RouteCostToToiletBetweenCorridorAndLivingSpaceList", "RouteCostToToiletBetweenLivingSpaceAndLivingSpaceList", "WaitTimeAcceptanceList", "WaitTimeMenBigToiletList", "WaitTimeMenSmallToiletList", "WaitTimeWomenToiletList"] #set log names of spot that you want to make excel sheet
    agent_log_names = [] #set log names of agent that you want to make excel sheet
    # set folder name you want to input data
    input_log_directory_names = ["10", "20", "30", "40", "50", "60", "70", "80", "90", "100", "110"]
    for input_log_directory_name in input_log_directory_names:
        readLogAndWriteExcels(spot_log_names, agent_log_names, input_log_directory_name)
    
def readLogAndWriteExcels(spot_log_names, agent_log_names, input_log_directory_name):
    all_log_directries = glob.glob("log_kagakuin/" + input_log_directory_name + "/*")
    log_dict = {}
    for (i, log_directory) in enumerate(all_log_directries):
        if len(spot_log_names) != 0:
            for spot_log_name in spot_log_names:
                if spot_log_name not in log_dict:
                    log_dict[spot_log_name]= []
                log_dict[spot_log_name].append(log_directory + "/spots/" + spot_log_name + ".log")
        if len(agent_log_names) != 0:
            for agent_log_name in agent_log_names:
                if agent_log_name not in log_dict:
                    log_dict[agent_log_name]= []
                log_dict[agent_log_name].append(log_directory + "/agents/" + agent_log_name + ".log")
    write_excels(log_dict, input_log_directory_name)
    
# [[], []]形式の場合で、全てを[2, 3, 4, 4]の形のリストに展開してから合計を算出するためのメソッド。平均は男女数に依存するのでexcelで行う
def unfold_list(cols):
    new_cols = []
    for i, col in enumerate(cols):
        if type(col) == int:
            continue
        if col.startswith('['):
            last_index = find_index_of_last_element(cols, i)
            if i is None:
                print("Error on find_index_of_last_element")
                return
            unfolded_list = cols[i:last_index]
            unfolded_list[0] = unfolded_list[0].replace('[', '')
            unfolded_list[-1] = unfolded_list[-1].replace(']', '')
            if unfolded_list[0] == '':
                continue
            unfolded_list = [int(i) for i in unfolded_list]
            new_cols+=unfolded_list
    return new_cols
    
# [[], []]形式の場合、リスト内の平均を算出して返す
def process_list_average(cols):
    new_cols = []
    for i, col in enumerate(cols):
        if type(col) == int:
            continue
        if col.startswith('['):
            last_index = find_index_of_last_element(cols, i)
            if i is None:
                print("Error by find_index_of_last_element")
                return
            list_to_calc_average = cols[i:last_index]
            list_to_calc_average[0] = list_to_calc_average[0].replace('[', '')
            list_to_calc_average[-1] = list_to_calc_average[-1].replace(']', '')
            if list_to_calc_average[0] == '':
                continue
            list_to_calc_average = [int(i) for i in list_to_calc_average]
            ave = np.average(list_to_calc_average)
            new_cols.append(ave)
    return new_cols
    
def find_index_of_last_element(cols, index):
#     i = index+1
    cols = cols[index:]
    for i, col in enumerate(cols):
        if col.endswith(']'):
            return index+i+1
    
def process_normal_log(col):
    if col.startswith('['):
        col = col[1:]
    if col.endswith(']'):
        col = col[:-1]
    return col
    
def replace_comma(ele):
    return ele.replace(',', '')
    
def write_excels(log_dict, input_log_directory_name):
    def read_log(path):
        log1 = codecs.open(path, mode="r", encoding="utf-8")
        log2 = codecs.open(path, mode="r", encoding="utf-8")
        row_num = sum(1 for row in log1)
        for (i, row) in enumerate(log2):
            if i+1 == row_num:
                line = row
        cols = line.split()
        cols = list(map(replace_comma, cols))
        new_cols = []
        firstCell = cols[2]
        
        if firstCell.startswith('[['):
            if os.path.basename(path).startswith('WaitTime'):
                unfolded_list = unfold_list(cols)
                #  合計を算出してエクセルに表記。np.sumのみのint型だとなぜかexcelに書き込む時にエラーが出るのでfloat64にキャストしてる
                return [np.float64(np.sum(unfolded_list))]
            return process_list_average(cols)
        for col in cols:
            new_col = process_normal_log(col)
            if new_col == '':
                continue
            new_cols.append(new_col)
            # エージェント数がおかしい時のロギング
#         if os.path.basename(path).startswith('AgentNum'):
#             for col in new_cols[2:]:
#                 print("new_cols. " + col)
        return new_cols[2:]

    book = px.Workbook()
    for key, value_list in log_dict.items():
        # RouteCostToToiletBetweenCorridorAndLivingSpaceList
        key = key.replace('Between', 'B_W')
        key = key.replace('Corridor', 'Cor')
        key = key.replace('LivingSpace', 'LS')
        key = key.replace('Acceptance', 'Accept')
        sheet = book.create_sheet(title=key[:30])
        for (i, value) in enumerate(value_list):
            row = read_log(value)
            row_num = i+1
            for (i, val) in enumerate(row):
                sheet.cell(row=row_num, column=i+1).value = val
    book.save('kagakuin_log_' + input_log_directory_name + '.xlsx')
    

main()


# if __name__ == "__main__":
#     main()

In [417]:
#  一度エクセルに整理したログデータをさらに1つのファイルに加工する
def process_main():
#     input_log_directory_names = ["100"]
    input_log_directory_names = ["10", "20", "30", "40", "50", "60", "70", "80", "90", "100", "110"]
    
    new_wb = px.Workbook()
    for dir_name in input_log_directory_names:
        read_wb = px.load_workbook('kagakuin_log_' + dir_name + '.xlsx')
        new_wb = processLog(new_wb, read_wb, dir_name)
    sheetnames = new_wb.get_sheet_names()
    for sheetname in sheetnames:
        if sheetname is 'Sheet':
            continue
        sheet = new_wb.get_sheet_by_name(sheetname)
        sheet = writeIndexToExcel(sheet)
    new_wb.save('kaiseki.xlsx')
    
def processLog(new_wb, read_wb, dir_name):
        tmp_wb = px.Workbook()
        sheetnames = new_wb.get_sheet_names()
        
        # AgentNum
        title = 'AgentNum'
        read_sheet = read_wb.get_sheet_by_name(title)
        if title in sheetnames:
            new_sheet = new_wb.get_sheet_by_name(title)
        else:
            new_sheet = new_wb.create_sheet(title=title)
        new_sheet = writeReshapeFromFirstColumnToLastRow(new_sheet, read_sheet)
    
        # make Invaded  Cost to toilet, routeCost - routeNum
        tmp_read_sheet = tmp_wb.create_sheet()
        tmp_read_sheet = subtractSheetBySheet(tmp_read_sheet, read_wb.get_sheet_by_name('RouteCostToToiletList'), read_wb.get_sheet_by_name('TotalRouteNumToToiletList'), int(dir_name))
        title = 'ICtoToilet'
        if title in sheetnames:
            new_sheet = new_wb.get_sheet_by_name(title)
        else:
            new_sheet = new_wb.create_sheet(title=title)
        new_sheet = writeReshapeFromFirstColumnToLastRow(new_sheet, tmp_read_sheet)

        # make Invaded  Cost to acceptance, routeCost - routeNum
        tmp_read_sheet = tmp_wb.create_sheet()
        tmp_read_sheet = subtractSheetBySheet(tmp_read_sheet, read_wb.get_sheet_by_name('RouteCostToAcceptList'), read_wb.get_sheet_by_name('TotalRouteNumToAcceptList'), int(dir_name))
        title = 'ICtoAcceptance'
        if title in sheetnames:
            new_sheet = new_wb.get_sheet_by_name(title)
        else:
            new_sheet = new_wb.create_sheet(title=title)
        new_sheet = writeReshapeFromFirstColumnToLastRow(new_sheet, tmp_read_sheet)
        
        def calcAndWrite(title):
            tmp_read_sheet = tmp_wb.create_sheet()
            tmp_read_sheet = read_wb.get_sheet_by_name(title)
            tmp_read_sheet = calcAveragePerAgent(tmp_read_sheet, read_sheet, int(dir_name))
            if title in sheetnames:
                new_sheet = new_wb.get_sheet_by_name(title)
            else:
                new_sheet = new_wb.create_sheet(title=title)
            new_sheet = writeReshapeFromFirstColumnToLastRow(new_sheet, tmp_read_sheet)
            
        # calcAndWrite execute
        calcAndWriteList = ['TotalRouteNumToAcceptList', 'RouteCostToAcceptList', 'TotalRouteNumToToiletList', 'RouteCostToToiletList']
        for title in calcAndWriteList:
            calcAndWrite(title)
            
        # 性別ごとの人数カウント。以下の形。
        #  sexCountDict = {
        #    10: [{man: 20, woman: 30}, {man: 20, woman: 30}],
         #   20: [{man: 20, woman: 30}, {man: 20, woman: 30}]
        #  }
        title = 'SexList'
        read_sheet = read_wb.get_sheet_by_name(title)
        if title in sheetnames:
            new_sheet = new_wb.get_sheet_by_name(title)
        else:
            new_sheet = new_wb.create_sheet(title=title)
        sexList = countEachSex(read_sheet)

        def calcAndWriteWaitTime(title, sexList, sex):
            read_sheet = read_wb.get_sheet_by_name(title)
            if title in sheetnames:
                new_sheet = new_wb.get_sheet_by_name(title)
            else:
                new_sheet = new_wb.create_sheet(title=title)
                new_sheet = calcWaitAverageAndWrite(new_sheet, read_sheet, sexList, sex)
            return new_sheet

        #  WaitTime
        calcAndWriteWaitTime('WaitTimeMenBigToiletList', sexList, 'man')
        calcAndWriteWaitTime('WaitTimeMenSmallToiletList', sexList, 'man')
        calcAndWriteWaitTime('WaitTimeWomenToiletList', sexList, 'woman')
            
        return new_wb
    
def countEachSex(read_sheet):
    sexList = []
    for row in read_sheet.rows:
        sexCount = {'man': 0, 'woman': 0}
        for cell in row:
            if cell.value == 'man':
                sexCount['man'] += 1
            elif cell.value == 'woman':
                sexCount['woman'] += 1
        sexList.append(sexCount)
    return sexList

# トイレの待ち時間平均を性別に応じて算出するメソッド
def calcWaitAverageAndWrite(new_sheet, read_sheet, sexList, sex):
    data = read_sheet.columns[0]
    last_row = new_sheet.max_row + 1
    for index, cell in enumerate(data):
        waitTimeAverage = np.float64(cell.value) / np.float64(sexList[index][sex])
        new_sheet.cell(row=last_row, column=index+2).value = waitTimeAverage
    return new_sheet


# 1列目を取得して、横に並べる。最終行に追加していく。
def writeReshapeFromFirstColumnToLastRow(new_sheet, read_sheet):
    data = read_sheet.columns[0]
    sheet_last_row_index = new_sheet.max_row + 1
    for index, d in enumerate(data):
        new_sheet.cell(row=sheet_last_row_index, column=index+2).value = np.float64(d.value)
    return new_sheet

# 行、列の番号を振り、罫線をつける 
def writeIndexToExcel(new_sheet):
    border_style = 'medium'
    sheet_last_row_index = new_sheet.max_column
    
    for i in np.arange(1, sheet_last_row_index+1):
        cell = new_sheet.cell(row=1, column=i+1)
        cell.value = np.float64(i)
        cell.border = Border(
            left=Side(border_style=border_style, color='FF000000'),
            right=Side(border_style=border_style, color='FF000000'),
            top=Side(border_style=border_style, color='FF000000'),
            bottom=Side(border_style=border_style, color='FF000000'),
            diagonal=Side(border_style=border_style, color='FF000000'),
            outline=Side(border_style=border_style, color='FF000000'),
            vertical=Side(border_style=border_style, color='FF000000'),
            horizontal=Side(border_style=border_style, color='FF000000')
        )
    for index, dir_name in enumerate(np.arange(10, 120, 10)):
        cell = new_sheet.cell(row=index+2, column=1)
        cell.value = str(dir_name) # TODO import from directory name
        cell.border = Border(
            left=Side(border_style=border_style, color='FF000000'),
            right=Side(border_style=border_style, color='FF000000'),
            top=Side(border_style=border_style, color='FF000000'),
            bottom=Side(border_style=border_style, color='FF000000'),
            diagonal=Side(border_style=border_style, color='FF000000'),
            outline=Side(border_style=border_style, color='FF000000'),
            vertical=Side(border_style=border_style, color='FF000000'),
            horizontal=Side(border_style=border_style, color='FF000000')
        )
    return new_sheet

# 第一引数側から第二引数側の、各セルの値を引いて、平均を算出して、一旦tmp_sheetの一列目に書き出し。
def subtractSheetBySheet(new_sheet, read_sheet1, read_sheet2, agentNum):
    for row in read_sheet1.rows:
        subtracted_list = []
        for cell in row:
            cellAt = cell.column + str(cell.row)
            subtracted_list.append(np.float64(cell.value) - np.float64(read_sheet2.cell(cellAt).value))
        last_row = new_sheet.max_row + 1
        new_sheet.cell(row=last_row, column=1).value = np.float64(np.sum(subtracted_list) / agentNum)
    return new_sheet

# 読み込んだシートの行ごとに平均を算出
def calcAveragePerAgent(new_sheet, read_sheet, agentNum):
    for row in read_sheet.rows:
        subtracted_list = []
        for cell in row:
            subtracted_list.append(np.float64(cell.value))
        last_row = new_sheet.max_row + 1
        new_sheet.cell(row=last_row, column=1).value = np.float64(np.sum(subtracted_list) / agentNum)
    return new_sheet
            
process_main()
#     sheet.cell(row=2, column=1)